# Data Scrapping

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

urls = pd.read_csv("weather_urls.csv", header=None)[0].tolist()

all_data = []

for BASE_URL in urls:
    print(f"🔹 Ambil data dari: {BASE_URL}")
    response = requests.get(BASE_URL)
    soup = BeautifulSoup(response.text, "html.parser")

    date_options = soup.select("#wt-his-select option")
    dates = [opt["value"] for opt in date_options]

    for date_code in dates:
        print(f"   📅 Scraping data tanggal {date_code} ...")
        url = f"{BASE_URL}&hd={date_code}"
        res = requests.get(url)
        soup = BeautifulSoup(res.text, "html.parser")

        table = soup.find("table", {"id": "wt-his"})
        if not table:
            print(f"   ⚠️ Tabel tidak ditemukan untuk {date_code}")
            continue

        rows = table.find("tbody").find_all("tr")
        for row in rows:
            th = row.find("th")
            tds = row.find_all("td")

            if not th or len(tds) < 8:
                continue

            time_str = th.text.strip().replace("\n", " ")
            temp = tds[1].text.strip()
            weather = tds[2].text.strip()
            wind_speed = tds[3].text.strip()
            wind_dir = tds[4].find("span")["title"] if tds[4].find("span") else ""
            humidity = tds[5].text.strip()
            barometer = tds[6].text.strip()
            visibility = tds[7].text.strip()

            all_data.append({
                "Date": date_code,
                "Time": time_str,
                "Temp": temp,
                "Weather": weather,
                "Wind_Speed": wind_speed,
                "Wind_Direction": wind_dir,
                "Humidity": humidity,
                "Barometer": barometer,
                "Visibility": visibility
            })

        time.sleep(2)


df = pd.DataFrame(all_data)
df.to_csv("weather_jakarta_Jan_to_Oct_20252.csv", index=False, encoding="utf-8-sig")

print("✅ Semua data dari Januari–Oktober 2025 sudah disimpan ke 'weather_jakarta_Jan_to_Oct_20252.csv'")

🔹 Ambil data dari: https://www.timeanddate.com/weather/indonesia/jakarta/historic?month=1&year=2025
   📅 Scraping data tanggal 20250101 ...
   📅 Scraping data tanggal 20250102 ...
   📅 Scraping data tanggal 20250103 ...
   📅 Scraping data tanggal 20250104 ...
   📅 Scraping data tanggal 20250105 ...
   📅 Scraping data tanggal 20250106 ...
   📅 Scraping data tanggal 20250107 ...
   📅 Scraping data tanggal 20250108 ...
   📅 Scraping data tanggal 20250109 ...
   📅 Scraping data tanggal 20250110 ...
   📅 Scraping data tanggal 20250111 ...
   📅 Scraping data tanggal 20250112 ...
   📅 Scraping data tanggal 20250113 ...
   📅 Scraping data tanggal 20250114 ...
   📅 Scraping data tanggal 20250115 ...
   📅 Scraping data tanggal 20250116 ...
   📅 Scraping data tanggal 20250117 ...
   📅 Scraping data tanggal 20250118 ...
   📅 Scraping data tanggal 20250119 ...
   📅 Scraping data tanggal 20250120 ...
   📅 Scraping data tanggal 20250121 ...
   📅 Scraping data tanggal 20250122 ...
   📅 Scraping data t

c:\Users\Lenovo E15\AppData\Local\Programs\Python\Python313\Lib\site-packages\bs4\builder\_htmlparser.py:174: RuntimeWarning: coroutine 'main' was never awaited
  attr_dict[key] = value


   📅 Scraping data tanggal 20250207 ...
   📅 Scraping data tanggal 20250208 ...
   📅 Scraping data tanggal 20250209 ...
   📅 Scraping data tanggal 20250210 ...
   📅 Scraping data tanggal 20250211 ...
   📅 Scraping data tanggal 20250212 ...
   📅 Scraping data tanggal 20250213 ...
   📅 Scraping data tanggal 20250214 ...
   📅 Scraping data tanggal 20250215 ...
   📅 Scraping data tanggal 20250216 ...
   📅 Scraping data tanggal 20250217 ...
   📅 Scraping data tanggal 20250218 ...
   📅 Scraping data tanggal 20250219 ...
   📅 Scraping data tanggal 20250220 ...
   📅 Scraping data tanggal 20250221 ...
   📅 Scraping data tanggal 20250222 ...
   📅 Scraping data tanggal 20250223 ...
   📅 Scraping data tanggal 20250224 ...
   📅 Scraping data tanggal 20250225 ...
   📅 Scraping data tanggal 20250226 ...
   📅 Scraping data tanggal 20250227 ...
   📅 Scraping data tanggal 20250228 ...
🔹 Ambil data dari: https://www.timeanddate.com/weather/indonesia/jakarta/historic?month=3&year=2025
   📅 Scraping data t

In [9]:
import pandas as pd

df = pd.read_csv('weather_jakarta_Jan_to_Oct_20252.csv', delimiter=';')
print(df)


          Date   Time   Temp          Weather Wind_Speed  \
0     20250101  00:00  26 °C  Passing clouds.     7 km/h   
1     20250101  01:00  26 °C  Passing clouds.     9 km/h   
2     20250101  02:00  26 °C  Passing clouds.     6 km/h   
3     20250101  03:00  26 °C  Passing clouds.     7 km/h   
4     20250101  04:00  25 °C  Passing clouds.     6 km/h   
...        ...    ...    ...              ...        ...   
6821  20251026  02:00  25 °C  Passing clouds.     4 km/h   
6822  20251026  03:00  25 °C  Passing clouds.     4 km/h   
6823  20251026  04:00  24 °C  Passing clouds.     4 km/h   
6824  20251026  05:00  24 °C  Passing clouds.     4 km/h   
6825  20251026  06:00  24 °C            Haze.     4 km/h   

                                         Wind_Direction Humidity  Barometer  \
0     Wind blowing from 240° West-southwest to East-...      87%  1010 mbar   
1     Wind blowing from 240° West-southwest to East-...      86%  1009 mbar   
2     Wind blowing from 250° West-southwes

In [10]:
df.head(5)

,Date,Time,Temp,Weather,Wind_Speed,Wind_Direction,Humidity,Barometer,Visibility
0,20250101,00:00,26 °C,Passing clouds.,7 km/h,Wind blowing from 240° West-southwest to East-...,87%,1010 mbar,5 km
1,20250101,01:00,26 °C,Passing clouds.,9 km/h,Wind blowing from 240° West-southwest to East-...,86%,1009 mbar,5 km
2,20250101,02:00,26 °C,Passing clouds.,6 km/h,Wind blowing from 250° West-southwest to East-...,86%,1009 mbar,4 km
3,20250101,03:00,26 °C,Passing clouds.,7 km/h,Wind blowing from 230° Southwest to Northeast,88%,1008 mbar,5 km
4,20250101,04:00,25 °C,Passing clouds.,6 km/h,Wind blowing from 230° Southwest to Northeast,90%,1008 mbar,5 km


# Data Cleaning

In [ ]:
import re

#change date to datetime format 
df['Date'] = df['Date'].astype(str)
df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%')